In [1]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

Mounted at /content/drive


In [8]:
my_path = '/content/drive/MyDrive/DeepL/Miniproject_1/'

In [3]:
import torch 
from torch import nn

In [4]:
def psnr(denoised, ground_truth):
    # Peak Signal to Noise Ratio : denoised and ground˙truth have range [0 , 1]
    mse = torch . mean ((denoised - ground_truth ) ** 2)
    return -10 * torch . log10 ( mse + 10** -8)

In [5]:
### For mini-project 1
class Model(nn.Module):
    def __init__(self) -> None:
        ## instantiate model + optimizer + loss function + any other stuff you need
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 5, stride = 1),  # N , 32, 28 , 28
            nn.ReLU(),
            nn.Conv2d(32,32,kernel_size=5,stride = 1, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2), # N , 32 , 14 , 14
            nn.Conv2d(32,64, kernel_size = 5, stride = 1, padding = 2), # N , 64, 10, 10
            nn.ReLU(),
            nn.Conv2d(64,64,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2 , stride = 2), # N , 64 , 5 , 5
            nn.Conv2d(64,128, kernel_size = 5, stride = 1) # N , 128 , 1 , 1
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128,64, kernel_size = 5, stride = 1), # N, 64, 5, 5
            nn.Upsample(scale_factor=2, mode = 'bilinear', align_corners=True), # N , 64, 10, 10
            nn.ReLU(),
            nn.ConvTranspose2d(64,64,kernel_size=5,stride=1, padding = 2), # N, 64, 10, 10
            nn.ReLU(),
            nn.ConvTranspose2d(64,32,kernel_size=5,stride=1), # N , 32, 14, 14
            nn.Upsample(scale_factor=2,mode = 'bilinear', align_corners=True), # N, 32, 28, 28
            nn.ReLU(),
            nn.ConvTranspose2d(32,32,kernel_size=5,stride=1, padding=2), # N, 32, 28, 28
            nn.ReLU(),
            nn.ConvTranspose2d(32,3,kernel_size=5,stride=1) # N, 3, 32, 32
        )

        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        

    def load_pretrained_model(self ) -> None:
        ## This loads the parameters saved in bestmodel . pth into the model
        pass

    def train(self, train_input, train_target) -> None:
        # : train˙input : tensor of size (N , C , H , W ) containing a noisy version of the images.
        # : train˙target : tensor of size (N , C , H , W ) containing another noisy version of the
        # same images , which only differs from the input by their noise.

        batch_size = 100
        epochs = 500
        total_loss = 0

        for epoch in range(epochs):
            print(f'Epoch {epoch}/{epochs-1} Training Loss {total_loss}')
            total_loss = 0
            for batch_input, batch_target in zip(train_input.split(batch_size), train_target.split(batch_size)):
                output = self.predict(batch_input)
                loss = self.criterion(output, batch_target)
                total_loss += loss
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
    
    def predict(self, test_input) -> torch.Tensor:
        # : test˙input : tensor of size ( N1 , C , H , W ) that has to be denoised by the trained
        # or the loaded network .
        # : returns a tensor of the size ( N1 , C , H , W )
        
        # like the forward method
        
        y = self.encoder(test_input)
        y = self.decoder(y)
        
        return y


In [12]:
def main():

    device = torch.device('cuda')

    # Load the data    
    print('Loading the data...')
    train_input, train_target = torch.load(my_path + 'train_data.pkl')
    test_input, test_target = torch.load(my_path + 'val_data.pkl')
    train_input, train_target = train_input.to(device), train_target.to(device)
    test_input, test_target = test_input.to(device), test_target.to(device)


    # Select a subset to speed up computations
    train_size = 1000
    train_input = train_input[:train_size]
    train_target = train_target[:train_size]
    test_input = test_input[:train_size]
    test_target = test_target[:train_size]


    # Convert the data into float type
    train_input = train_input.float()
    train_target = train_target.float()
    test_input = test_input.float()
    test_target = test_target.float()

    
    print(f'Training data of size {train_input.shape}')

    # Defining and training the model
    model = Model()
    model = model.to(device)
    print('Training the model...')
    model.train(train_input, train_target)

    # Testing
    print('Using the trained model to denoise validation images...')
    with torch.no_grad():
        prediction = model.predict(test_input)

    # Evaluating error
    error = psnr(prediction, test_target)
    print(f'The PSNR on the validation set is {error} DB')

In [13]:
main()

Loading the data...
Training data of size torch.Size([1000, 3, 32, 32])
Training the model...
Epoch 0/499 Training Loss 0
Epoch 1/499 Training Loss 163230.296875
Epoch 2/499 Training Loss 143448.984375
Epoch 3/499 Training Loss 89303.59375
Epoch 4/499 Training Loss 69913.1875
Epoch 5/499 Training Loss 58672.859375
Epoch 6/499 Training Loss 50199.37890625
Epoch 7/499 Training Loss 45319.0
Epoch 8/499 Training Loss 41292.1328125
Epoch 9/499 Training Loss 38351.53125
Epoch 10/499 Training Loss 36208.9375
Epoch 11/499 Training Loss 34502.20703125
Epoch 12/499 Training Loss 33538.140625
Epoch 13/499 Training Loss 32948.7421875
Epoch 14/499 Training Loss 32371.7109375
Epoch 15/499 Training Loss 32120.2578125
Epoch 16/499 Training Loss 36821.14453125
Epoch 17/499 Training Loss 34216.2890625
Epoch 18/499 Training Loss 30961.90234375
Epoch 19/499 Training Loss 29223.736328125
Epoch 20/499 Training Loss 28241.814453125
Epoch 21/499 Training Loss 27732.037109375
Epoch 22/499 Training Loss 27435.1

KeyboardInterrupt: ignored